In [ ]:
import os
import sys
import cv2
from glob import glob

In [ ]:
#Si no está en colab, puedes ignorar (no comentar)
PATH_COLAB = "/content/drive/MyDrive/Smartech/Basura/extract_frames"

In [ ]:

is_colab = 'google.colab' in sys.modules

if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = PATH_COLAB
    if os.path.exists(BASE_PATH):
        os.chdir(BASE_PATH)
        print("📂 Ahora trabajando en:", os.getcwd())
    else:
        print(f"Error: The directory '{BASE_PATH}' does not exist. Please verify the path.")
else:
    print("No está en Colab")

In [ ]:
def extract_frames(video_path, output_folder, fps=2):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Cargar el video
    video = cv2.VideoCapture(video_path)

    # Verificar si se abrió correctamente
    if not video.isOpened():
        print(f"No se pudo abrir el video: {video_path}")
        return

    # Obtener la cantidad total de frames y el FPS original del video
    original_fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calcular el intervalo de frames para extraer
    interval = max(1, int(original_fps / fps))

    frame_number = 0  # Número del frame actual
    extracted_frames = 0  # Contador de frames extraídos

    while True:
        ret, frame = video.read()
        if not ret:  # Salir si no hay más frames
            break

        # Guardar el frame si está en el intervalo
        if frame_number % interval == 0:
            # Generar el nombre de archivo y guardar la imagen
            video_name = os.path.splitext(os.path.basename(video_path))[0]
            view_folder = os.path.basename(os.path.dirname(video_path))
            video_output_folder = f"{view_folder}_{video_name}"
            video_output_folder = os.path.join(output_folder, video_output_folder)
            if not os.path.exists(video_output_folder):
                os.makedirs(video_output_folder)
            output_path = os.path.join(video_output_folder, f"{str(view_folder)}_{str(video_name)}_{extracted_frames:04d}.jpg")
            cv2.imwrite(output_path, frame)  # Guardar el frame
            extracted_frames += 1

        frame_number += 1

    # Liberar el video
    video.release()
    print(f"Se extrajeron {extracted_frames} frames de '{video_path}' y se guardaron en '{video_output_folder}'.")


In [ ]:
"""
Puedes configurar si quieres solo un subdirectorio o varios (leer comentarios)
"""
def process_all_videos(parent_folder, output_folder, fps=4):
    # Buscar todos los archivos de video en la carpeta padre y subcarpetas
    #video_files = glob(os.path.join(parent_folder, "*.mp4"), recursive=True) # solo un nivel
    video_files = glob(os.path.join(parent_folder, "**", "*.mp4"), recursive=True) # varios niveles de subdirectorios

    if not video_files:
        print("No se encontraron archivos de video en la carpeta proporcionada.")
        return

    for video_path in video_files:
        print(f"Procesando video: {video_path}")
        extract_frames(video_path, output_folder, fps=fps)


In [ ]:
output_folder = "data"
parent_folder = "../../"  # Carpeta padre con videos y subcarpetas
# output_folder = os.path.join(data_folder, parent_folder)
process_all_videos(parent_folder, output_folder, fps=1)